In [ ]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset

import random
from tqdm.auto import tqdm
from PIL import Image
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt
import os
import json
import pandas as pd
from monai.inferers import sliding_window_inference
import warnings
warnings.filterwarnings('ignore')
import PIL
PIL.Image.MAX_IMAGE_PIXELS = 933120000000
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

def seed_everything(seed=123):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

In [ ]:
def read_rgb_img(img_path):
    img= np.array(Image.open(img_path))
    orign_shape= img.shape[:2][::-1]
    return img, orign_shape

def get_test_transform():
    return A.Compose([
        ToTensorV2(p=1.0),
    ])
def get_pad_transform(img_size):
    return A.Compose([
        A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0, p=1),
    ])
def get_crop_transform(img_size):
    return A.Compose([
        A.CenterCrop(img_size, img_size, p=1),
    ])

class Customize_Dataset(Dataset):
    def __init__(self, df, transforms):
        self.df = df
        self.image_path = df['image_path'].values
        self.transforms = transforms
    
    def __getitem__(self, index):
        img_path = self.image_path[index]
        img, ori_shape= read_rgb_img(img_path)
        
        ## scale adjust
        img_size= min(img.shape[0], img.shape[1])
        img_size= img_size/CFG['img_scale']
        img_size= int(img_size)
        img= cv2.resize(img, (img_size, img_size))
        
        if img.shape[0]<CFG['window_size']:
            pad= img.shape[0]
            aug= get_pad_transform(CFG['window_size'])
            img= aug(image= img)['image']
        else:
            pad= 0
        
        img = self.transforms(image=img)["image"]
        return {
            'img_path': img_path,
            'image': torch.tensor(img/255, dtype=torch.float32),
            'ori_shape': torch.tensor(ori_shape),
            'pad': pad,
        }
    
    def __len__(self):
        return len(self.df)

In [ ]:
import sys
sys.path.append('pretrained/coat')
from coat import *
from daformer import *
from helper import *
from pretrained.coat.coatnet_inference import *

In [ ]:
class customize_model(nn.Module):
    def __init__(self, model_name):
        super(customize_model, self).__init__()
        
    def forward(self, images):
        out= self.model(images)['logits']
        return out

# CFG

In [ ]:
CFG= {
    'fold': 0,
    'img_scale': (4/(0.4945/0.4)),
    'window_size': 768,
    'TTA': False,
    'model': None,
    'show_result': False,
}
# CFG['model']= f"./train_model/model_cv{CFG['fold']}_best.pth"
# CFG['model']= f"./train_model/model_cv{CFG['fold']}_ep99.pth"
# CFG['model']= f"./test_model/coat_m_s4_w768_0.48/model_cv{CFG['fold']}_best.pth"
CFG['model']= f"./test_model/coat_m_s4_w768_0.48"

# CFG['model']= [torch.load(CFG['model'], map_location= 'cuda:0')]
CFG['model']= [torch.load(m, map_location= 'cuda:0') for m in glob.glob(f"{CFG['model']}/**/*pth", recursive=True)[:5]]

In [ ]:
df= pd.read_csv('./Data/sample_submission.csv')
df['image_path']= None
for i in range(len(df)):
    id_= df.loc[i, ['id']].values[0]
    path= f'./Data/test_images/{id_}.tiff'
    df.loc[i, ['image_path']]= path
    
test_dataset= Customize_Dataset(df, get_test_transform())
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)
df.head(3)

# Inference

In [ ]:
def rle_encode(img):
    #the image should be transposed
    pixels = img.T.flatten()
    # This simplified method requires first and last pixel to be zero
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def inference(model, img):
    img= img.cuda()
    for i, m in enumerate(model):
        with torch.no_grad():
            m.eval()
            if CFG['TTA']:
                imgs= torch.cat([img, 
                                 img.rot90(1, [2,3]),
                                 img.rot90(2, [2,3]),
                                 img.rot90(3, [2,3]),
                                 img.flip(-1), 
                                 img.flip(-1).rot90(1, [2,3]),
                                 img.flip(-1).rot90(2, [2,3]),
                                 img.flip(-1).rot90(3, [2,3])], dim=0)
                pred= sliding_window_inference(imgs, 
                                               (CFG['window_size'], CFG['window_size']), 
                                               sw_batch_size= 2, 
                                               predictor= m,
                                               mode= 'gaussian',
                                               overlap= 0.25)
                pred= (pred[0] + 
                       pred[1].rot90(-1, [1,2]) +
                       pred[2].rot90(-2, [1,2]) +
                       pred[3].rot90(-3, [1,2]) +
                       pred[4].flip(-1) + 
                       pred[5].rot90(-1, [1,2]).flip(-1) + 
                       pred[6].rot90(-2, [1,2]).flip(-1) + 
                       pred[7].rot90(-3, [1,2]).flip(-1)) / 8
            else:
                pred= sliding_window_inference(img, 
                                               (CFG['window_size'], CFG['window_size']), 
                                               sw_batch_size= 2, 
                                               predictor= m,
                                               mode= 'gaussian',
                                               overlap= 0.25)[0]
        if pred.shape[0]!=1:
            if i==0: preds= pred.softmax(dim=0)
            else: preds+= pred.softmax(dim=0)
        else:
            if i==0: preds= pred.sigmoid()
            else: preds+= pred.sigmoid()
    pred= preds/len(model)
    pred= pred.cpu().permute(1,2,0).numpy()
    return pred

In [ ]:
indx= 0
for i, data in enumerate(tqdm(test_loader)):
    for j in range(len(data['image'])):
        img_path= data['img_path'][j]
        img= data['image'][j]
        ori_shape= data['ori_shape'][j].numpy()
        pad= data['pad'][j]
        
        ## inference
        img= torch.unsqueeze(img, dim= 0)
        pred_mask= inference(CFG['model'], img)
        
        if pred_mask.shape[2]!=1:
            pred_mask= pred_mask[..., 2]
        
        ## if pad
        if pad!=0:
            aug= get_crop_transform(pad)
            pred_mask= aug(image= pred_mask)['image']
            pred_mask= cv2.resize(pred_mask, tuple(ori_shape))
        else:
            pred_mask= cv2.resize(pred_mask, tuple(ori_shape))
        
        thr= 0.2
        pred_mask[pred_mask>=thr]= 1
        pred_mask[pred_mask<thr]= 0
        pred_mask= pred_mask.astype(np.uint8)
        
        sample_img= img[0].permute(1,2,0).numpy()
        if pad!=0:
            aug= get_crop_transform(pad)
            sample_img= aug(image= sample_img)['image']
            sample_img= cv2.resize(sample_img, tuple(ori_shape))
        else:
            sample_img= cv2.resize(sample_img, tuple(ori_shape))
            
        plt.imshow(sample_img)
        plt.show()
        plt.imshow(pred_mask)
        plt.show()

#         pred_mask*=255
#         pred_mask= pred_mask.astype(np.uint8)
#         im= Image.fromarray(pred_mask)
#         im.save(img_path.replace('img_png', 'mask_png'))
        
        rle= rle_encode(pred_mask)
        df.loc[indx, 'rle']= rle
        indx+= 1
df